- earlystopingを早める
- 

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import xgboost as xgb
%matplotlib inline


In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
test_df

,kind,date,area
0,だいこん,20220502,千葉_各地_青森
1,だいこん,20220506,千葉_各地_青森
2,だいこん,20220507,千葉_各地_青森
3,だいこん,20220509,千葉_各地_青森
4,だいこん,20220510,千葉_各地_青森
...,...,...,...
315,ミニトマト,20220526,各地_愛知
316,ミニトマト,20220527,各地_愛知
317,ミニトマト,20220528,各地_愛知
318,ミニトマト,20220530,各地_愛知


In [4]:
weather

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
204315,20220426,21.3,23.6,2022/4/26 14:27,17.9,2022/4/26 04:17,2.5,0.0,89.0,東京
204316,20220427,23.0,27.7,2022/4/27 13:12,18.9,2022/4/28 0:00,0.5,1.6,79.0,東京
204317,20220428,16.9,21.1,2022/4/28 15:03,14.2,2022/4/28 22:40,0.0,5.5,57.0,東京
204318,20220429,12.9,16.4,2022/4/29 09:21,9.0,2022/4/29 23:55,40.0,0.2,86.0,東京


In [5]:
sample

,kind,date,mode_price
0,だいこん,20220502,0
1,だいこん,20220506,0
2,だいこん,20220507,0
3,だいこん,20220509,0
4,だいこん,20220510,0
...,...,...,...
315,ミニトマト,20220526,0
316,ミニトマト,20220527,0
317,ミニトマト,20220528,0
318,ミニトマト,20220530,0


In [6]:
#profile = ProfileReport(train_df, title="Profiling Report")
#profile.to_notebook_iframe()
#profile.to_file("your_report.html")
# amount削除
train_df = train_df.drop(columns=['amount'])


In [7]:
def Data_cre(df):
    # --- area を分解して複数行に展開 ---
    df["area_list"] = df["area"].str.split("_")

    # explodeで行を増やす
    df = df.explode("area_list")
    df = df.drop(columns=["area"])
    df = df.rename(columns={"area_list": "area"})

    # --- 日付フォーマット揃え ---
    df["date"] = df["date"].astype(str)
    df["date"] = pd.to_datetime(df["date"])

    # --- weather と結合 ---
    df = pd.merge(df, weather, on=["date", "area"], how="left")

    # --- 年月日列を追加 ---
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day

    # --- max_temp_time / min_temp_time 分解 ---
    df["max_temp_time"] = pd.to_datetime(df["max_temp_time"])
    df["maxtemp_year"] = df["max_temp_time"].dt.year
    df["maxtemp_month"] = df["max_temp_time"].dt.month
    df["maxtemp_day"] = df["max_temp_time"].dt.day
    df["maxtemp_hour"] = df["max_temp_time"].dt.hour

    df["min_temp_time"] = pd.to_datetime(df["min_temp_time"])
    df["mintemp_year"] = df["min_temp_time"].dt.year
    df["mintemp_month"] = df["min_temp_time"].dt.month
    df["mintemp_day"] = df["min_temp_time"].dt.day
    df["mintemp_hour"]= df["min_temp_time"].dt.hour

    # --- 不要列削除 ---
    df = df.drop(columns=["date", "max_temp_time", "min_temp_time"])

    # --- category 型に変換 ---
    df["kind"] = df["kind"].astype("category")
    df["area"] = df["area"].astype("category")

    return df
weather["date"] = pd.to_datetime(weather["date"])

train_df = Data_cre(train_df)

test_df = Data_cre(test_df) 



In [8]:
train_df["area"]

0         千葉
1         千葉
2         各地
3         青森
4         千葉
          ..
212120    長野
212121    新潟
212122    長野
212123    新潟
212124    長野
Name: area, Length: 212125, dtype: category
Categories (39, object): ['アメリカ', 'カナダ', 'トンガ', 'ニュージーランド', ..., '静岡', '香川', '高知', '鹿児島']

In [9]:
train_df

,kind,mode_price,area,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,year,month,day,maxtemp_year,maxtemp_month,maxtemp_day,maxtemp_hour,mintemp_year,mintemp_month,mintemp_day,mintemp_hour
0,だいこん,735.0,千葉,NaN,NaN,NaN,NaN,NaN,NaN,2005,11,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,だいこん,840.0,千葉,NaN,NaN,NaN,NaN,NaN,NaN,2005,11,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,だいこん,840.0,各地,NaN,NaN,NaN,NaN,NaN,NaN,2005,11,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,だいこん,840.0,青森,NaN,NaN,NaN,NaN,NaN,NaN,2005,11,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,だいこん,735.0,千葉,NaN,NaN,NaN,NaN,NaN,NaN,2005,11,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212120,しめじ,48.5,長野,NaN,NaN,NaN,NaN,NaN,NaN,2017,5,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212121,しめじ,48.5,新潟,NaN,NaN,NaN,NaN,NaN,NaN,2017,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212122,しめじ,48.5,長野,NaN,NaN,NaN,NaN,NaN,NaN,2017,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212123,しめじ,43.0,新潟,NaN,NaN,NaN,NaN,NaN,NaN,2017,5,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
X = train_df.drop(['mode_price'], axis=1)
y = train_df['mode_price']
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [11]:
### chatgpt
from sklearn.model_selection import KFold
import numpy as np
import lightgbm as lgb

models = {}  # kind → model list or averaged は辞書に保存

K = 5  # ← Fold数

for kd, group in train_df.groupby("kind", observed=False):

    print(f"\n===== {kd} モデルをKFold({K})で学習中 =====")

    X = group.drop(columns=["mode_price"],axis=1)
    y = group["mode_price"]

    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    fold_models = []  # foldごとのモデルを保存

    for fold, (train_index, valid_index) in enumerate(kf.split(X)):
        print(f"Fold {fold+1}/{K}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        train_data = lgb.Dataset(X_train, y_train)
        valid_data = lgb.Dataset(X_valid, y_valid)

        model = xgb.XGBRegressor(
            n_estimators=10,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=fold,
            early_stopping_rounds = 10,
            enable_categorical=True
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=True
        )
        
        fold_models.append(model)

    models[kd] = fold_models  # kindごとにfoldモデルを保存



===== いんげん モデルをKFold(5)で学習中 =====
Fold 1/5
[0]	validation_0-rmse:576.38670
[1]	validation_0-rmse:570.12798
[2]	validation_0-rmse:563.16477
[3]	validation_0-rmse:560.19194
[4]	validation_0-rmse:555.05758
[5]	validation_0-rmse:550.05255
[6]	validation_0-rmse:546.18750
[7]	validation_0-rmse:544.21518
[8]	validation_0-rmse:536.91473
[9]	validation_0-rmse:533.93956
Fold 2/5
[0]	validation_0-rmse:556.44404
[1]	validation_0-rmse:549.74437
[2]	validation_0-rmse:546.28469
[3]	validation_0-rmse:541.37619
[4]	validation_0-rmse:536.25273
[5]	validation_0-rmse:528.93244
[6]	validation_0-rmse:523.34188
[7]	validation_0-rmse:522.97486
[8]	validation_0-rmse:519.15999
[9]	validation_0-rmse:512.86130
Fold 3/5
[0]	validation_0-rmse:531.74062
[1]	validation_0-rmse:525.11178
[2]	validation_0-rmse:521.95466
[3]	validation_0-rmse:518.79947
[4]	validation_0-rmse:515.91101
[5]	validation_0-rmse:508.41622
[6]	validation_0-rmse:507.03601
[7]	validation_0-rmse:504.84123
[8]	validation_0-rmse:498.12975
[9]	valida

In [12]:
models

{'いんげん': [XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=10,
               enable_categorical=True, eval_metric=None, feature_types=None,
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=0.05, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=10,
               n_jobs=None, num_parallel_tree=None, ...),
  XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=10,
               enable_catego

In [13]:
import numpy as np

pred_list = []

for kd, group in test_df.groupby("kind"):

    fold_models = models[kd]  # ← foldモデルが複数入っている
    X_test = group.copy()

    # 全 fold モデルの予測を行い平均
    fold_pred = np.mean([m.predict(X_test) for m in fold_models], axis=0)

    X_test["mode_price"] = fold_pred
    pred_list.append(X_test)

# まとめ・並び順合わせ
result = pd.concat(pred_list).sort_index()

# 日付整形（year,month,day → YYYYMMDD）
result["date"] = (
    result["year"].astype(str)
    + result["month"].astype(str).str.zfill(2)
    + result["day"].astype(str).str.zfill(2)
)
result["date"] = result["date"].astype(int)

# 提出形式に整形
submit = result[["kind", "date", "mode_price"]]
submit.to_csv("submit.csv", index=False)

submit

C:\Users\ND013\AppData\Local\Temp\ipykernel_20240\1180845613.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for kd, group in test_df.groupby("kind"):


,kind,date,mode_price
0,だいこん,20220502,982.273438
1,だいこん,20220502,982.273438
2,だいこん,20220502,982.273438
3,だいこん,20220506,981.211121
4,だいこん,20220506,981.211121
...,...,...,...
815,ミニトマト,20220528,130.391373
816,ミニトマト,20220530,130.613876
817,ミニトマト,20220530,130.613876
818,ミニトマト,20220531,130.724655


In [14]:
submit_mean = (
    submit.groupby(["kind", "date"], as_index=False)["mode_price"]
          .mean()
)
# ① area 側の kind の出現順を取得
kind_order = test_df["kind"].drop_duplicates().tolist()

# ② price 側の kind を Categorical に変換して順序を固定
submit_mean["kind"] = pd.Categorical(submit_mean["kind"], categories=kind_order, ordered=True)

# ③ 並び替え
df_price_sorted = submit_mean.sort_values(["kind", "date"]).reset_index(drop=True)

C:\Users\ND013\AppData\Local\Temp\ipykernel_20240\975664683.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  submit.groupby(["kind", "date"], as_index=False)["mode_price"]


In [15]:
df_price_sorted.to_csv("submit.csv", index=False)
df_price_sorted

,kind,date,mode_price
0,だいこん,20220502,982.273438
1,だいこん,20220506,981.211121
2,だいこん,20220507,979.266602
3,だいこん,20220509,980.990540
4,だいこん,20220510,981.435242
...,...,...,...
315,ミニトマト,20220526,130.245041
316,ミニトマト,20220527,130.245041
317,ミニトマト,20220528,130.391373
318,ミニトマト,20220530,130.613876
